In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm
from scipy import stats
from datetime import datetime
import matplotlib.pyplot as plt
import statsmodels.api as sm
import patsy

from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import pmdarima as pm
import seaborn as sns
from scipy.stats import zscore

In [2]:
df = pd.read_excel("Clapham MSC Final Analysis (1).xlsx", parse_dates = [0]).rename(columns={"Row Labels":"date"})
df = df.iloc[0:-1]

In [3]:
df["date"] = pd.to_datetime(df["date"])

In [4]:
df

,date,TNOs,MSC Numbers,isWeekend,Day of Week,Drugs,Robbery,Sexual Offences,Theft and Handling,VAP,Average Temp,Average Humidity,Average Wind Speed
0,2018-02-01,8,0,0.0,4.0,2,0,0,4,1,5.70,70.8350,11.20
1,2018-02-03,16,4,1.0,6.0,0,2,0,11,2,4.25,86.5550,4.75
2,2018-05-25,10,8,1.0,5.0,0,1,0,7,1,16.95,83.8355,5.30
3,2018-06-29,8,9,1.0,5.0,0,1,0,4,2,18.45,67.5165,9.30
4,2018-07-07,11,1,1.0,6.0,0,0,0,3,5,23.25,57.0280,6.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,2018-07-27,10,17,1.0,5.0,0,0,0,4,4,20.60,76.2955,7.45
201,2018-07-29,1,0,0.0,7.0,0,0,0,1,0,20.10,81.8545,10.95
202,2017-12-09,1,0,1.0,6.0,0,0,0,1,0,2.85,69.7560,6.00
203,2018-07-30,5,0,0.0,1.0,0,1,0,3,0,19.75,66.5200,7.95


In [5]:
dates = pd.bdate_range("2018-01-26", "2018-07-28", freq="D")
dates

DatetimeIndex(['2018-01-26', '2018-01-27', '2018-01-28', '2018-01-29',
               '2018-01-30', '2018-01-31', '2018-02-01', '2018-02-02',
               '2018-02-03', '2018-02-04',
               ...
               '2018-07-19', '2018-07-20', '2018-07-21', '2018-07-22',
               '2018-07-23', '2018-07-24', '2018-07-25', '2018-07-26',
               '2018-07-27', '2018-07-28'],
              dtype='datetime64[ns]', length=184, freq='D')

In [6]:

dateframe = pd.DataFrame(index=dates)

dateframe["DayOfWeek"] = dateframe.index.weekday.astype("category")

dateframe["date"] = dateframe.index


In [7]:
dateframe

,DayOfWeek,date
2018-01-26,4,2018-01-26
2018-01-27,5,2018-01-27
2018-01-28,6,2018-01-28
2018-01-29,0,2018-01-29
2018-01-30,1,2018-01-30
...,...,...
2018-07-24,1,2018-07-24
2018-07-25,2,2018-07-25
2018-07-26,3,2018-07-26
2018-07-27,4,2018-07-27


In [8]:
combined = dateframe.merge(df, how="left", on="date")
combined

,DayOfWeek,date,TNOs,MSC Numbers,isWeekend,Day of Week,Drugs,Robbery,Sexual Offences,Theft and Handling,VAP,Average Temp,Average Humidity,Average Wind Speed
0,4,2018-01-26,1.0,0.0,1.0,5.0,0.0,1.0,0.0,0.0,0.0,7.35,76.7720,3.55
1,5,2018-01-27,2.0,0.0,1.0,6.0,0.0,0.0,0.0,1.0,1.0,11.10,86.3940,10.60
2,6,2018-01-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,2018-01-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2018-01-30,3.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,9.25,81.7195,10.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,1,2018-07-24,4.0,0.0,0.0,2.0,1.0,0.0,0.0,1.0,2.0,25.15,52.7300,6.40
181,2,2018-07-25,3.0,0.0,0.0,3.0,0.0,0.0,0.0,2.0,0.0,23.05,76.5360,9.75
182,3,2018-07-26,5.0,0.0,0.0,4.0,0.0,0.0,1.0,4.0,0.0,28.25,45.1545,6.10
183,4,2018-07-27,10.0,17.0,1.0,5.0,0.0,0.0,0.0,4.0,4.0,20.60,76.2955,7.45


In [9]:
combined.isna().sum()

DayOfWeek             0
date                  0
TNOs                  2
MSC Numbers           2
isWeekend             2
Day of Week           2
Drugs                 2
Robbery               2
Sexual Offences       2
Theft and Handling    2
VAP                   2
Average Temp          3
Average Humidity      3
Average Wind Speed    3
dtype: int64

We now have a complete dataset of 185 rows, with only a few missing values.  Let's deal with them reasonably.


In [10]:
combined.columns

Index(['DayOfWeek', 'date', 'TNOs', 'MSC Numbers', 'isWeekend', 'Day of Week',
       'Drugs', 'Robbery', 'Sexual Offences', 'Theft and Handling', 'VAP',
       'Average Temp', 'Average Humidity', 'Average Wind Speed'],
      dtype='object')

In [11]:
combined[[ 'TNOs', 'MSC Numbers', 'isWeekend', 'Day of Week',
       'Drugs', 'Robbery', 'Sexual Offences', 'Theft and Handling', 'VAP',
       'Average Temp', 'Average Humidity', 'Average Wind Speed']] = combined[[ 'TNOs', 'MSC Numbers', 'isWeekend', 'Day of Week',
       'Drugs', 'Robbery', 'Sexual Offences', 'Theft and Handling', 'VAP',
       'Average Temp', 'Average Humidity', 'Average Wind Speed']].interpolate()

combined.isna().sum()

DayOfWeek             0
date                  0
TNOs                  0
MSC Numbers           0
isWeekend             0
Day of Week           0
Drugs                 0
Robbery               0
Sexual Offences       0
Theft and Handling    0
VAP                   0
Average Temp          0
Average Humidity      0
Average Wind Speed    0
dtype: int64

Now we have interpolated and have no missing value - a full 185 day dataset



In [12]:
combined.rename(columns={"MSC Numbers":"OfficerDosage", "Sexual Offences":"SexualOffences",
                         "Theft and Handling":"Theft",'Average Temp':"Temp",
                'Average Humidity':"Humid", 'Average Wind Speed':"Wind"}, inplace=True)


In [13]:
combined.columns

Index(['DayOfWeek', 'date', 'TNOs', 'OfficerDosage', 'isWeekend',
       'Day of Week', 'Drugs', 'Robbery', 'SexualOffences', 'Theft', 'VAP',
       'Temp', 'Humid', 'Wind'],
      dtype='object')

In [14]:
for col in ['TNOs', 'OfficerDosage', 'isWeekend',
       'Day of Week', 'Drugs', 'Robbery', 'SexualOffences', 'Theft', 'VAP',
       'Temp', 'Humid', 'Wind']:
    new_name = "z_" + col
    combined[new_name] = zscore(combined[col])

combined

,DayOfWeek,date,TNOs,OfficerDosage,isWeekend,Day of Week,Drugs,Robbery,SexualOffences,Theft,...,z_isWeekend,z_Day of Week,z_Drugs,z_Robbery,z_SexualOffences,z_Theft,z_VAP,z_Temp,z_Humid,z_Wind
0,4,2018-01-26,1.000000,0.0,1.000000,5.000000,0.0,1.0,0.0,0.000000,...,1.526464,0.497042,-0.474193,1.358845,-0.349727,-1.065823,-1.052367,-0.803641,0.464384,-1.471496
1,5,2018-01-27,2.000000,0.0,1.000000,6.000000,0.0,0.0,0.0,1.000000,...,1.526464,1.005068,-0.474193,-0.560135,-0.349727,-0.724686,-0.491307,-0.229538,1.288659,0.885904
2,6,2018-01-28,2.333333,0.0,0.666667,4.666667,0.0,0.0,0.0,1.333333,...,0.796759,0.327700,-0.474193,-0.560135,-0.349727,-0.610973,-0.678327,-0.323946,1.155177,0.869185
3,0,2018-01-29,2.666667,0.0,0.333333,3.333333,0.0,0.0,0.0,1.666667,...,0.067054,-0.349669,-0.474193,-0.560135,-0.349727,-0.497261,-0.865347,-0.418354,1.021696,0.852466
4,1,2018-01-30,3.000000,0.0,0.000000,2.000000,0.0,0.0,0.0,2.000000,...,-0.662651,-1.027037,-0.474193,-0.560135,-0.349727,-0.383549,-1.052367,-0.512762,0.888215,0.835747
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,1,2018-07-24,4.000000,0.0,0.000000,2.000000,1.0,0.0,0.0,1.000000,...,-0.662651,-1.027037,0.987903,-0.560135,-0.349727,-0.724686,0.069753,1.921436,-1.595189,-0.518504
181,2,2018-07-25,3.000000,0.0,0.000000,3.000000,0.0,0.0,0.0,2.000000,...,-0.662651,-0.519011,-0.474193,-0.560135,-0.349727,-0.383549,-1.052367,1.599938,0.444167,0.601678
182,3,2018-07-26,5.000000,0.0,0.000000,4.000000,0.0,0.0,1.0,4.000000,...,-0.662651,-0.010984,-0.474193,-0.560135,2.238255,0.298725,-1.052367,2.396029,-2.244148,-0.618819
183,4,2018-07-27,10.000000,17.0,1.000000,5.000000,0.0,0.0,0.0,4.000000,...,1.526464,0.497042,-0.474193,-0.560135,-0.349727,0.298725,1.191874,1.224858,0.423564,-0.167402


In [19]:
feats = ["OfficerDosage",'TNOs', 'Drugs', 'Robbery', 'SexualOffences', 'Theft', 'VAP',
       'Temp', 'Humid', 'Wind']

In [20]:
from statsmodels.tsa.stattools import kpss
def kpss_test(series, **kw):
    statistic, p_value, n_lags, critical_values = kpss(series, **kw)
    # Format Output
    print(f'KPSS Statistic: {statistic}')
    print(f'p-value: {p_value}')
    print(f'num lags: {n_lags}')
    print('Critial Values:')
    for key, value in critical_values.items():
        print(f'   {key} : {value}')
    print(f'Result: The series is {"not " if p_value < 0.05 else ""}stationary')

In [21]:
for feat in feats:
    print(feat)
    kpss_test(combined[feat])

OfficerDosage
KPSS Statistic: 0.09378097013771428
p-value: 0.1
num lags: 14
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is stationary
TNOs
KPSS Statistic: 0.583988481171663
p-value: 0.02409195625712154
num lags: 14
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is not stationary
Drugs
KPSS Statistic: 0.3946578508328111
p-value: 0.07945782291689178
num lags: 14
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is stationary
Robbery
KPSS Statistic: 0.12363623461912183
p-value: 0.1
num lags: 14
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is stationary
SexualOffences
KPSS Statistic: 0.17184998577871624
p-value: 0.1
num lags: 14
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is stationary
Theft
KPSS Statistic: 0.1397951535757444
p-value: 0.1
num lags: 14
Cr

c:\users\admin\dropbox\data projects\nte_hotspot_policing\venv\lib\site-packages\statsmodels\tsa\stattools.py:1875: FutureWarning: The behavior of using nlags=None will change in release 0.13.Currently nlags=None is the same as nlags="legacy", and so a sample-size lag length is used. After the next release, the default will change to be the same as nlags="auto" which uses an automatic lag length selection method. To silence this warning, either use "auto" or "legacy"
  warnings.warn(msg, FutureWarning)
c:\users\admin\dropbox\data projects\nte_hotspot_policing\venv\lib\site-packages\statsmodels\tsa\stattools.py:1910: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warnings.warn(
c:\users\admin\dropbox\data projects\nte_hotspot_policing\venv\lib\site-packages\statsmodels\tsa\stattools.py:1910: InterpolationWarning: The test statistic is outside of the range of p-values

This looks a lot better - most of our features are appropriately not stationary.


In [35]:
result_df = pd.DataFrame()
crimes = ['TNOs', 'Drugs', 'Robbery', 'SexualOffences', 'Theft', 'VAP']


for crime in crimes:
    auto = pm.auto_arima(combined[crime])
    predictions = auto.predict_in_sample()
    mse = mean_squared_error(combined[crime], predictions)
    result_df.loc["Order", crime+"_fit"] = str(auto.to_dict()['order'])
    result_df.loc["SeasonOrder", crime+"_fit"] = str(auto.to_dict()['seasonal_order'])
    result_df.loc["AIC", crime+"_fit"] = auto.aic()
    result_df.loc["BIC", crime+"_fit"] = auto.bic()
    result_df.loc["MSE", crime+"_fit"] = mse
    result_df.loc["RMSE", crime+"_fit"] = str(np.sqrt(mse))

result_df

,TNOs_fit,Drugs_fit,Robbery_fit,SexualOffences_fit,Theft_fit,VAP_fit
Order,"(5, 1, 1)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(5, 0, 0)","(0, 0, 0)"
SeasonOrder,"(0, 0, 0, 0)","(0, 0, 0, 0)","(0, 0, 0, 0)","(0, 0, 0, 0)","(0, 0, 0, 0)","(0, 0, 0, 0)"
AIC,1089.037665,388.454831,287.843599,177.182218,907.717696,742.840274
BIC,1111.542215,394.895543,294.284311,183.62293,930.260187,749.280986
MSE,19.897872,0.467787,0.271556,0.149306,7.330848,3.176736
RMSE,4.460703108476523,0.6839493443410436,0.5211102380538392,0.38640142704133007,2.707553882834275,1.7823399379954128


In [85]:
result_df = pd.DataFrame()
crimes = ['TNOs', 'Drugs', 'Robbery', 'SexualOffences', 'Theft', 'VAP']

#set m to 7
for crime in crimes:
    auto = pm.auto_arima(combined[crime], m=7)
    predictions = auto.predict_in_sample()
    mse = mean_squared_error(combined[crime], predictions)
    result_df.loc["Order", crime+"_fit"] = str(auto.to_dict()['order'])
    result_df.loc["SeasonOrder", crime+"_fit"] = str(auto.to_dict()['seasonal_order'])
    result_df.loc["AIC", crime+"_fit"] = auto.aic()
    result_df.loc["BIC", crime+"_fit"] = auto.bic()
    result_df.loc["MSE", crime+"_fit"] = mse
    result_df.loc["RMSE", crime+"_fit"] = str(np.sqrt(mse))

result_df

,TNOs_fit,Drugs_fit,Robbery_fit,SexualOffences_fit,Theft_fit,VAP_fit
Order,"(2, 1, 1)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(2, 0, 2)"
SeasonOrder,"(1, 0, 2, 7)","(0, 0, 0, 7)","(2, 0, 1, 7)","(0, 0, 0, 7)","(2, 0, 2, 7)","(2, 0, 2, 7)"
AIC,1051.175975,388.454831,284.866755,177.182218,871.897256,738.335512
BIC,1076.895461,394.895543,300.968534,183.62293,891.219391,770.53907
MSE,16.043317,0.467787,0.259253,0.149306,6.125175,2.820435
RMSE,4.005410986965652,0.6839493443410436,0.5091687192114839,0.38640142704133007,2.4749090362578396,1.6794149832294623


You have to explicitly set you seasonal term!  THis kind of changes everything.



In [90]:
def calc_auto_m(patsy_string):
    """takes a Patsy string in the given format, returns a dataframe for all crime types"""

    result_df = pd.DataFrame()
    crimes = ['TNOs', 'Drugs', 'Robbery', 'SexualOffences', 'Theft', 'VAP']
    for crime in crimes:
        string_arima = crime + " ~ 0 + " + patsy_string
        y, X = patsy.dmatrices(string_arima, data=combined, return_type="dataframe")
        auto = pm.auto_arima(y,exogenous=X, m=7)
        #result_df[crime+"_params"] = auto.to_dict()['params'].round(3)
        try:
            X = X.drop(columns="C(Saturday)[False]")
        except:
            pass
        try:
            X = X.drop(columns="C(Friday)[False]")
        except:
            pass
        for value in auto.to_dict()['params'].keys():
            result_df.loc[value, crime+"_params"] = auto.to_dict()['params'][value]
        result_df[crime+"_p"] = auto.to_dict()['pvalues'].round(3)
        predictions = auto.predict_in_sample(exogenous=X)
        mse = mean_squared_error(y, predictions)
        result_df.loc["Order", crime+"_fit"] = str(auto.to_dict()['order'])
        result_df.loc["SeasonOrder", crime+"_fit"] = str(auto.to_dict()['seasonal_order'])
        result_df.loc["AIC", crime+"_fit"] = auto.aic()
        result_df.loc["BIC", crime+"_fit"] = auto.bic()
        result_df.loc["MSE", crime+"_fit"] = mse
        result_df.loc["RMSE", crime+"_fit"] = str(np.sqrt(mse))

    return result_df

In [93]:

calc_auto_m("OfficerDosage*C(Saturday) + C(Friday) + C(Sunday) + Temp")

,TNOs_params,TNOs_p,TNOs_fit,Drugs_params,Drugs_p,Drugs_fit,Robbery_params,Robbery_p,Robbery_fit,SexualOffences_params,SexualOffences_p,SexualOffences_fit,Theft_params,Theft_p,Theft_fit,VAP_params,VAP_p,VAP_fit
C(Saturday)[0.0],4.062390,0.000,NaN,0.123185,0.485,NaN,0.226462,0.043,NaN,0.023394,0.842,NaN,2.009320,0.000,NaN,1.097622,0.003,NaN
C(Saturday)[1.0],12.805921,0.000,NaN,-0.067816,0.835,NaN,0.729005,0.000,NaN,-0.111712,0.731,NaN,7.860544,0.000,NaN,2.772370,0.000,NaN
C(Friday)[T.1.0],1.223133,0.349,NaN,-0.015200,0.972,NaN,-0.072871,0.694,NaN,-0.041079,0.925,NaN,1.128556,0.177,NaN,0.064139,0.932,NaN
C(Sunday)[T.1.0],0.153829,0.888,NaN,-0.060399,0.688,NaN,-0.126503,0.471,NaN,0.143052,0.097,NaN,0.443510,0.526,NaN,-0.064105,0.882,NaN
OfficerDosage,0.342609,0.012,NaN,-0.003951,0.941,NaN,0.048377,0.002,NaN,0.007855,0.867,NaN,0.171964,0.085,NaN,0.101743,0.139,NaN
OfficerDosage:C(Saturday)[T.1.0],-0.448244,0.039,NaN,0.074295,0.318,NaN,-0.068120,0.046,NaN,0.073734,0.281,NaN,-0.305141,0.028,NaN,-0.094335,0.470,NaN
Temp,0.071129,0.093,NaN,0.016348,0.095,NaN,-0.001973,0.725,NaN,0.005067,0.252,NaN,-0.007954,0.757,NaN,0.032549,0.104,NaN
sigma2,12.487494,0.000,NaN,0.449707,0.000,NaN,0.230342,0.000,NaN,0.136524,0.000,NaN,4.855435,0.000,NaN,2.701535,0.000,NaN
Order,NaN,NaN,"(0, 0, 0)",NaN,NaN,"(0, 0, 0)",NaN,NaN,"(0, 0, 0)",NaN,NaN,"(0, 0, 0)",NaN,NaN,"(0, 0, 0)",NaN,NaN,"(0, 0, 0)"
SeasonOrder,NaN,NaN,"(0, 0, 0, 7)",NaN,NaN,"(0, 0, 0, 7)",NaN,NaN,"(0, 0, 0, 7)",NaN,NaN,"(0, 0, 0, 7)",NaN,NaN,"(0, 0, 0, 7)",NaN,NaN,"(0, 0, 0, 7)"


In [87]:
calc_auto_m("OfficerDosage + Temp")

,TNOs_params,TNOs_p,TNOs_fit,Drugs_params,Drugs_p,Drugs_fit,Robbery_params,Robbery_p,Robbery_fit,SexualOffences_params,SexualOffences_p,SexualOffences_fit,Theft_params,Theft_p,Theft_fit,VAP_params,VAP_p,VAP_fit
intercept,1.486638,0.159,NaN,NaN,NaN,NaN,0.235584,0.022,NaN,NaN,NaN,NaN,0.298200,0.129,NaN,1.218231,0.000,NaN
OfficerDosage,0.485506,0.000,NaN,0.007614,0.623,NaN,0.045860,0.000,NaN,0.012926,0.070,NaN,0.181238,0.014,NaN,0.133402,0.000,NaN
Temp,0.028475,0.673,NaN,0.022958,0.000,NaN,-0.001768,0.754,NaN,0.008185,0.002,NaN,-0.019860,0.640,NaN,0.034038,0.079,NaN
ar.L1,-0.800349,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ar.L2,-0.803472,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ma.L1,0.817135,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ma.L2,0.961392,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ar.S.L7,0.897986,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.047422,0.545,NaN,NaN,NaN,NaN
ma.S.L7,-0.723007,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.041318,0.699,NaN,NaN,NaN,NaN
sigma2,14.480832,0.000,NaN,0.458316,0.000,NaN,0.246772,0.000,NaN,0.147002,0.000,NaN,5.775510,0.000,NaN,2.917131,0.000,NaN


In [88]:
calc_auto_m("Temp ")

,TNOs_params,TNOs_p,TNOs_fit,Drugs_params,Drugs_p,Drugs_fit,Robbery_params,Robbery_p,Robbery_fit,SexualOffences_params,SexualOffences_p,SexualOffences_fit,Theft_params,Theft_p,Theft_fit,VAP_params,VAP_p,VAP_fit
intercept,1.410353,0.122,NaN,NaN,NaN,NaN,0.019534,0.390,NaN,NaN,NaN,NaN,0.151096,0.134,NaN,0.050326,0.617,NaN
Temp,-0.022834,0.757,NaN,0.023780,0.0,NaN,-0.002579,0.724,NaN,0.00958,0.0,NaN,-0.037049,0.356,NaN,0.031709,0.161,NaN
ar.L1,-0.830292,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ar.L2,-0.857407,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ma.L1,0.829118,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ma.L2,0.951651,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ar.S.L7,0.925065,0.000,NaN,NaN,NaN,NaN,0.817598,0.000,NaN,NaN,NaN,NaN,0.958382,0.000,NaN,0.965564,0.000,NaN
ma.S.L7,-0.691193,0.000,NaN,NaN,NaN,NaN,-0.852894,0.000,NaN,NaN,NaN,NaN,-0.777290,0.000,NaN,-0.910180,0.000,NaN
sigma2,15.478456,0.000,NaN,0.459031,0.0,NaN,0.255749,0.000,NaN,0.14906,0.0,NaN,6.059753,0.000,NaN,2.985206,0.000,NaN
Order,NaN,NaN,"(2, 0, 2)",NaN,NaN,"(0, 0, 0)",NaN,NaN,"(0, 0, 0)",NaN,NaN,"(0, 0, 0)",NaN,NaN,"(0, 0, 0)",NaN,NaN,"(0, 0, 0)"


In [89]:
calc_auto_m("OfficerDosage")


,TNOs_params,TNOs_p,TNOs_fit,Drugs_params,Drugs_p,Drugs_fit,Robbery_params,Robbery_p,Robbery_fit,SexualOffences_params,SexualOffences_p,SexualOffences_fit,Theft_params,Theft_p,Theft_fit,VAP_params,VAP_p,VAP_fit
intercept,0.001601,0.513,NaN,0.316288,0.001,NaN,0.213347,0.006,NaN,0.115269,0.092,NaN,0.845950,0.132,NaN,1.646439,0.0,NaN
OfficerDosage,0.358603,0.001,NaN,0.004691,0.796,NaN,0.045837,0.000,NaN,0.011591,0.116,NaN,0.205355,0.004,NaN,0.133829,0.0,NaN
ar.L1,-0.020148,0.789,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.772200,0.000,NaN,NaN,NaN,NaN
ar.L2,0.136544,0.086,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.877654,0.000,NaN,NaN,NaN,NaN
ma.L1,-0.985396,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.853866,0.000,NaN,NaN,NaN,NaN
ar.S.L7,0.191573,0.432,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.027311,0.699,NaN,NaN,NaN,NaN
ar.S.L14,0.690964,0.003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.856341,0.000,NaN,NaN,NaN,NaN
ma.S.L7,-0.097304,0.722,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.034120,0.768,NaN,NaN,NaN,NaN
ma.S.L14,-0.473658,0.049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.631422,0.000,NaN,NaN,NaN,NaN
sigma2,14.424073,0.000,NaN,0.467522,0.000,NaN,0.246903,0.000,NaN,0.147725,0.000,NaN,5.573194,0.000,NaN,2.966706,0.0,NaN


In [29]:
features = ['OfficerDosage',  'Temp' , 'Humid' , 'Wind']

auto = pm.auto_arima(combined['TNOs'],exogenous=combined[['OfficerDosage']])


auto.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                  185
Model:               SARIMAX(0, 1, 1)   Log Likelihood                -530.658
Date:                Mon, 05 Jul 2021   AIC                           1067.317
Time:                        16:56:24   BIC                           1076.962
Sample:                             0   HQIC                          1071.226
                                - 185                                         
Covariance Type:                  opg                                         
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
OfficerDosage     0.5746      0.077      7.451      0.000       0.423       0.726
ma.L1            -0.9739      0.020    -49.032      0.000      -1.013      -0.935
sigma2           18.4310      1.121     16.438      0.000      16.233      20.629
===================================================================================
Ljung-Box (L1) (Q):                   0.43   Jarque-Bera (JB):               165.04
Prob(Q):                              0.51   Prob(JB):                         0.00
Heteroskedasticity (H):               0.89   Skew:                             1.49
Prob(H) (two-sided):                  0.66   Kurtosis:                         6.56
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

So the problem is officer dosage is perfectly correlating with our seasonality, and so it's getting killed off.

What I'll do instead is take the TNO order I had above, and THEN add our officer details.  That should show us if even with covariance, we have a significante effect.

In [44]:
auto = pm.auto_arima(y= combined['TNOs'])
auto.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                  185
Model:               SARIMAX(5, 1, 1)   Log Likelihood                -537.519
Date:                Mon, 05 Jul 2021   AIC                           1089.038
Time:                        17:17:19   BIC                           1111.542
Sample:                             0   HQIC                          1098.159
                                - 185                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.0108      0.083     -0.131      0.896      -0.173       0.151
ar.L2         -0.0931      0.098     -0.951      0.342      -0.285       0.099
ar.L3         -0.2243      0.079     -2.843      0.004      -0.379      -0.070
ar.L4         -0.1986      0.100     -1.978      0.048      -0.395      -0.002
ar.L5         -0.1932      0.086     -2.258      0.024      -0.361      -0.026
ma.L1         -0.9343      0.031    -29.766      0.000      -0.996      -0.873
sigma2        19.8007      1.574     12.577      0.000      16.715      22.886
===================================================================================
Ljung-Box (L1) (Q):                   0.04   Jarque-Bera (JB):                85.38
Prob(Q):                              0.85   Prob(JB):                         0.00
Heteroskedasticity (H):               0.96   Skew:                             1.24
Prob(H) (two-sided):                  0.86   Kurtosis:                         5.24
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [41]:
model = ARIMA(combined['TNOs'], order=(5,1,1), exog=combined[['OfficerDosage', 'Temp']], dates=dates)
model_fit = model.fit()
model_fit.summary()

c:\users\admin\dropbox\data projects\nte_hotspot_policing\venv\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                   TNOs   No. Observations:                  185
Model:                 ARIMA(5, 1, 1)   Log Likelihood                -528.545
Date:                Mon, 05 Jul 2021   AIC                           1075.090
Time:                        17:14:45   BIC                           1104.025
Sample:                             0   HQIC                          1086.818
                                - 185                                         
Covariance Type:                  opg                                         
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
OfficerDosage     0.5205      0.096      5.400      0.000       0.332       0.709
Temp              0.0875      0.048      1.818      0.069      -0.007       0.182
ar.L1            -0.0256      0.089     -0.287      0.774      -0.201       0.149
ar.L2             0.0536      0.097      0.553      0.580      -0.136       0.244
ar.L3            -0.0831      0.089     -0.937      0.349      -0.257       0.091
ar.L4            -0.0741      0.113     -0.654      0.513      -0.296       0.148
ar.L5            -0.0736      0.105     -0.699      0.484      -0.280       0.133
ma.L1            -0.9999      4.312     -0.232      0.817      -9.452       7.452
sigma2           17.7523     75.937      0.234      0.815    -131.082     166.586
===================================================================================
Ljung-Box (L1) (Q):                   0.03   Jarque-Bera (JB):               188.80
Prob(Q):                              0.87   Prob(JB):                         0.00
Heteroskedasticity (H):               0.88   Skew:                             1.53
Prob(H) (two-sided):                  0.62   Kurtosis:                         6.91
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

Let's contrast this to a linear model where we know what day of the week it is.




In [47]:
combined

,DayOfWeek,date,TNOs,OfficerDosage,isWeekend,Day of Week,Drugs,Robbery,SexualOffences,Theft,...,z_isWeekend,z_Day of Week,z_Drugs,z_Robbery,z_SexualOffences,z_Theft,z_VAP,z_Temp,z_Humid,z_Wind
0,4,2018-01-26,1.000000,0.0,1.000000,5.000000,0.0,1.0,0.0,0.000000,...,1.526464,0.497042,-0.474193,1.358845,-0.349727,-1.065823,-1.052367,-0.803641,0.464384,-1.471496
1,5,2018-01-27,2.000000,0.0,1.000000,6.000000,0.0,0.0,0.0,1.000000,...,1.526464,1.005068,-0.474193,-0.560135,-0.349727,-0.724686,-0.491307,-0.229538,1.288659,0.885904
2,6,2018-01-28,2.333333,0.0,0.666667,4.666667,0.0,0.0,0.0,1.333333,...,0.796759,0.327700,-0.474193,-0.560135,-0.349727,-0.610973,-0.678327,-0.323946,1.155177,0.869185
3,0,2018-01-29,2.666667,0.0,0.333333,3.333333,0.0,0.0,0.0,1.666667,...,0.067054,-0.349669,-0.474193,-0.560135,-0.349727,-0.497261,-0.865347,-0.418354,1.021696,0.852466
4,1,2018-01-30,3.000000,0.0,0.000000,2.000000,0.0,0.0,0.0,2.000000,...,-0.662651,-1.027037,-0.474193,-0.560135,-0.349727,-0.383549,-1.052367,-0.512762,0.888215,0.835747
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,1,2018-07-24,4.000000,0.0,0.000000,2.000000,1.0,0.0,0.0,1.000000,...,-0.662651,-1.027037,0.987903,-0.560135,-0.349727,-0.724686,0.069753,1.921436,-1.595189,-0.518504
181,2,2018-07-25,3.000000,0.0,0.000000,3.000000,0.0,0.0,0.0,2.000000,...,-0.662651,-0.519011,-0.474193,-0.560135,-0.349727,-0.383549,-1.052367,1.599938,0.444167,0.601678
182,3,2018-07-26,5.000000,0.0,0.000000,4.000000,0.0,0.0,1.0,4.000000,...,-0.662651,-0.010984,-0.474193,-0.560135,2.238255,0.298725,-1.052367,2.396029,-2.244148,-0.618819
183,4,2018-07-27,10.000000,17.0,1.000000,5.000000,0.0,0.0,0.0,4.000000,...,1.526464,0.497042,-0.474193,-0.560135,-0.349727,0.298725,1.191874,1.224858,0.423564,-0.167402


In [54]:
combined["DayofWeek"] = combined['date'].dt.weekday

combined.loc[combined["DayofWeek"]==4, 'Friday'] = 1
combined.loc[combined["DayofWeek"]==5, 'Saturday'] = 1
combined.loc[combined["DayofWeek"]==6, 'Sunday'] = 1
combined[["Friday", "Saturday", "Sunday"]] = combined[["Friday", "Saturday", "Sunday"]].fillna(0)

In [57]:
import statsmodels.formula.api as smf

mod = smf.ols(formula='TNOs ~ C(DayofWeek)*OfficerDosage + Temp', data=combined)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   TNOs   R-squared:                       0.442
Model:                            OLS   Adj. R-squared:                  0.410
Method:                 Least Squares   F-statistic:                     13.80
Date:                Mon, 05 Jul 2021   Prob (F-statistic):           9.45e-18
Time:                        17:29:00   Log-Likelihood:                -495.21
No. Observations:                 185   AIC:                             1012.
Df Residuals:                     174   BIC:                             1048.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept                           4.0340      0.886      4.553      0.000       2.285       5.783
C(DayofWeek)[T.1]                  -0.6683      1.006     -0.664      0.508      -2.655       1.318
C(DayofWeek)[T.2]                   0.5401      1.007      0.537      0.592      -1.447       2.527
C(DayofWeek)[T.3]                   0.2451      1.006      0.244      0.808      -1.741       2.231
C(DayofWeek)[T.4]                   1.2531      1.588      0.789      0.431      -1.881       4.387
C(DayofWeek)[T.5]                   8.7729      1.418      6.188      0.000       5.975      11.571
C(DayofWeek)[T.6]                   0.1813      1.016      0.178      0.859      -1.825       2.187
OfficerDosage                       0.1563      0.931      0.168      0.867      -1.681       1.994
C(DayofWeek)[T.1]:OfficerDosage -7.483e-17    3.6e-16     -0.208      0.836   -7.86e-16    6.37e-16
C(DayofWeek)[T.2]:OfficerDosage  8.542e-17    9.3e-16      0.092      0.927   -1.75e-15    1.92e-15
C(DayofWeek)[T.3]:OfficerDosage  1.853e-17   1.48e-17      1.251      0.213   -1.07e-17    4.78e-17
C(DayofWeek)[T.4]:OfficerDosage     0.1862      0.938      0.198      0.843      -1.666       2.038
C(DayofWeek)[T.5]:OfficerDosage    -0.2619      0.950     -0.276      0.783      -2.138       1.614
C(DayofWeek)[T.6]:OfficerDosage     0.2320      2.785      0.083      0.934      -5.264       5.728
Temp                                0.0711      0.041      1.732      0.085      -0.010       0.152
==============================================================================
Omnibus:                       28.805   Durbin-Watson:                   1.885
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               62.739
Skew:                           0.705   Prob(JB):                     2.38e-14
Kurtosis:                       5.481   Cond. No.                     6.96e+19
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 7.89e-36. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [64]:
import statsmodels.formula.api as smf

mod = smf.ols(formula='TNOs ~ + C(Friday) + C(Saturday) + C(Sunday) + OfficerDosage + Temp', data=combined)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   TNOs   R-squared:                       0.431
Model:                            OLS   Adj. R-squared:                  0.415
Method:                 Least Squares   F-statistic:                     27.08
Date:                Mon, 05 Jul 2021   Prob (F-statistic):           2.38e-20
Time:                        17:32:13   Log-Likelihood:                -497.11
No. Observations:                 185   AIC:                             1006.
Df Residuals:                     179   BIC:                             1026.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              4.0739      0.619      6.586      0.000       2.853       5.295
C(Friday)[T.1.0]       2.1352      1.311      1.629      0.105      -0.451       4.721
C(Saturday)[T.1.0]     7.5115      0.936      8.021      0.000       5.664       9.359
C(Sunday)[T.1.0]       0.1585      0.792      0.200      0.842      -1.405       1.722
OfficerDosage          0.2239      0.138      1.620      0.107      -0.049       0.496
Temp                   0.0702      0.041      1.725      0.086      -0.010       0.151
==============================================================================
Omnibus:                       29.316   Durbin-Watson:                   1.913
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               63.532
Skew:                           0.719   Prob(JB):                     1.60e-14
Kurtosis:                       5.485   Cond. No.                         78.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [68]:
crimes

['TNOs', 'Drugs', 'Robbery', 'SexualOffences', 'Theft', 'VAP']

In [71]:
mod = smf.ols(formula='SexualOffences ~ + C(Friday) + C(Saturday)*OfficerDosage + C(Sunday) + Temp', data=combined)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         SexualOffences   R-squared:                       0.086
Model:                            OLS   Adj. R-squared:                  0.055
Method:                 Least Squares   F-statistic:                     2.776
Date:                Mon, 05 Jul 2021   Prob (F-statistic):             0.0132
Time:                        17:34:06   Log-Likelihood:                -78.317
No. Observations:                 185   AIC:                             170.6
Df Residuals:                     178   BIC:                             193.2
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                            0.0233      0.064      0.361      0.718      -0.104       0.151
C(Friday)[T.1.0]                    -0.0411      0.152     -0.270      0.787      -0.341       0.259
C(Saturday)[T.1.0]                  -0.1350      0.133     -1.018      0.310      -0.397       0.127
C(Sunday)[T.1.0]                     0.1430      0.083      1.732      0.085      -0.020       0.306
OfficerDosage                        0.0079      0.017      0.468      0.640      -0.025       0.041
C(Saturday)[T.1.0]:OfficerDosage     0.0737      0.033      2.258      0.025       0.009       0.138
Temp                                 0.0051      0.004      1.196      0.233      -0.003       0.013
==============================================================================
Omnibus:                      132.865   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              910.091
Skew:                           2.839   Prob(JB):                    2.38e-198
Kurtosis:                      12.265   Cond. No.                         80.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [77]:
result_df = pd.DataFrame()

for crime in crimes:
    mod = smf.ols(formula=crime + ' ~ + C(Friday) + C(Saturday)*OfficerDosage + C(Sunday) + Temp', data=combined)
    res = mod.fit()
    result_df[crime+"_p"] = res.pvalues.round(3)
    result_df[crime+"_param"] = res.params.round(3)


In [78]:
result_df

,TNOs_p,TNOs_param,Drugs_p,Drugs_param,Robbery_p,Robbery_param,SexualOffences_p,SexualOffences_param,Theft_p,Theft_param,VAP_p,VAP_param
Intercept,0.000,4.062,0.294,0.123,0.008,0.226,0.718,0.023,0.000,2.009,0.000,1.098
C(Friday)[T.1.0],0.401,1.223,0.956,-0.015,0.712,-0.073,0.787,-0.041,0.215,1.129,0.925,0.064
C(Saturday)[T.1.0],0.000,8.744,0.429,-0.191,0.004,0.503,0.310,-0.135,0.000,5.851,0.005,1.675
C(Sunday)[T.1.0],0.846,0.154,0.687,-0.060,0.240,-0.127,0.085,0.143,0.369,0.443,0.862,-0.064
OfficerDosage,0.034,0.343,0.897,-0.004,0.028,0.048,0.640,0.008,0.088,0.172,0.175,0.102
C(Saturday)[T.1.0]:OfficerDosage,0.153,-0.448,0.212,0.074,0.110,-0.068,0.025,0.074,0.119,-0.305,0.517,-0.094
Temp,0.081,0.071,0.035,0.016,0.722,-0.002,0.233,0.005,0.754,-0.008,0.086,0.033


In [63]:

combined[['OfficerDosage','DayOfWeek']].sort_values(by='OfficerDosage').iloc[-30:-1]

,OfficerDosage,DayOfWeek
120,5.0,5
113,5.0,5
149,5.0,5
85,6.0,5
15,6.0,5
42,6.0,4
35,6.0,4
56,6.0,4
184,7.0,5
71,7.0,5
